# imports

In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import *

from Model import HourglassNet
from Data_utils import DataGenerator

## model

In [2]:
model = HourglassNet(num_classes=10, 
                     num_stacks=1, 
                     num_channels=128, 
                     inres=(128,128), 
                     outres=(32,32)).get_model()

In [3]:
optimizer = tf.keras.optimizers.Adam(lr=5e-4)
loss = tf.keras.metrics.mean_squared_error

model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
front_conv_1x1_x1 (Conv2D)      (None, 64, 64, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 64)   256         front_conv_1x1_x1[0][0]          
__________________________________________________________________________________________________
front_residual_x1_conv_1x1_x1 ( (None, 64, 64, 32)   2080        batch_normalization[0][0]        
__________________________________________________________________________________________________
batch_norm

## datasets

In [5]:
train_gen = DataGenerator('dataset/', 'train_landmarks.csv', 
                          inres=(128,128), outres=(32,32), nparts=10, is_train=True)
valid_gen = DataGenerator('dataset/', 'valid_landmarks.csv', 
                          inres=(128,128), outres=(32,32), nparts=10)
test_gen = DataGenerator('dataset/', 'test_landmarks.csv', 
                         inres=(128,128), outres=(32,32), nparts=10)

## callbacks

In [6]:
my_call = [
    ModelCheckpoint('model/model.h5', save_best_only=True),
    EarlyStopping(patience=20),
    TensorBoard(log_dir='train/'),
    ReduceLROnPlateau(),
    CSVLogger('train/history.csv')
]

## training

In [7]:
batch_size = 32

model.fit_generator(train_gen.generator(batch_size, 1), len(train_gen)//batch_size,
                    validation_data=valid_gen.generator(batch_size, 1), 
                    validation_steps=len(valid_gen)//batch_size,
                    epochs=10000, 
                    callbacks=my_call)

Epoch 1/10000
63/63 [==============================] - 22s 354ms/step - loss: 10.0644 - acc: 0.0973 - val_loss: 2.0920 - val_acc: 0.0725
Epoch 2/10000
63/63 [==============================] - 4s 63ms/step - loss: 0.8740 - acc: 0.1057 - val_loss: 3.2296 - val_acc: 0.1974
Epoch 3/10000
63/63 [==============================] - 4s 63ms/step - loss: 0.5035 - acc: 0.1103 - val_loss: 3.8304 - val_acc: 0.1706
Epoch 4/10000
63/63 [==============================] - 4s 64ms/step - loss: 0.3479 - acc: 0.1175 - val_loss: 5.1834 - val_acc: 0.1390
Epoch 5/10000
63/63 [==============================] - 4s 63ms/step - loss: 0.2724 - acc: 0.1224 - val_loss: 4.5390 - val_acc: 0.0970
Epoch 6/10000
63/63 [==============================] - 4s 63ms/step - loss: 0.2294 - acc: 0.1263 - val_loss: 5.6183 - val_acc: 0.0978
Epoch 7/10000
63/63 [==============================] - 5s 72ms/step - loss: 0.2028 - acc: 0.1318 - val_loss: 1.5238 - val_acc: 0.0672
Epoch 8/10000
63/63 [==============================] - 4s 6

Epoch 62/10000
63/63 [==============================] - 5s 72ms/step - loss: 0.0754 - acc: 0.2163 - val_loss: 0.0745 - val_acc: 0.1981
Epoch 63/10000
63/63 [==============================] - 5s 72ms/step - loss: 0.0746 - acc: 0.2191 - val_loss: 0.0738 - val_acc: 0.1992
Epoch 64/10000
63/63 [==============================] - 4s 63ms/step - loss: 0.0741 - acc: 0.2208 - val_loss: 0.0741 - val_acc: 0.2069
Epoch 65/10000
63/63 [==============================] - 5s 72ms/step - loss: 0.0734 - acc: 0.2215 - val_loss: 0.0737 - val_acc: 0.1985
Epoch 66/10000
63/63 [==============================] - 5s 72ms/step - loss: 0.0728 - acc: 0.2225 - val_loss: 0.0733 - val_acc: 0.2048
Epoch 67/10000
63/63 [==============================] - 4s 63ms/step - loss: 0.0730 - acc: 0.2197 - val_loss: 0.0741 - val_acc: 0.2178
Epoch 68/10000
63/63 [==============================] - 5s 72ms/step - loss: 0.0718 - acc: 0.2200 - val_loss: 0.0722 - val_acc: 0.2143
Epoch 69/10000
63/63 [==============================] -

## evaluation

In [8]:
model.evaluate_generator(test_gen.generator(1,1), steps=len(test_gen))

[0.002380064756835054, 0.1788330078125]